In [8]:
import pandas as pd
from med_words import med_words

In [3]:
wrong = pd.read_csv('all_wrong.csv')

In [7]:
g.analyze_numbers(wrong)

AttributeError: 'Pandas' object has no attribute 'ContactSICDesc'

In [19]:
def clean_zipcode(zipcode):
    zipcode=str(zipcode)
    zipcode = zipcode.replace(".0","")
    if len(zipcode)==3:
        zipcode = "00"+zipcode
    if len(zipcode)==4: 
        zipcode = "0"+zipcode
    else:
        zipcode = zipcode[0:5]
    return(zipcode)
    
def is_healthy(name):
    healthy = False
    for word in med_words:
        if word in str(name).lower():
            healthy = True
    return(healthy)

addr_translate = {
    'DR': 'DRIVE',
    'PKWY': 'PARKWAY',
    'RD': 'ROAD',
    'ST': 'STREET',
    'STE': 'SUITE',
    'N': 'NORTH',
    'S': 'SOUTH',
    'E': 'EAST',
    'W': 'WEST',
    'LN': 'LANE',
    'CTR': 'CENTER',
    'CT': 'COURT',
    'BLVD': 'BOULEVARD',
    'CIR': 'CIRCLE',
    'HTS': 'HEIGHTS',
    'AVE': 'AVENUE',
    'HWY': 'HIGHWAY',
    'JCT': 'JUNCTION',
    'LK': 'LAKE',
    'MTN': 'MTN',
    'APT': 'APARTMENT',
    'RM': 'ROOM',
    'PL': 'PLACE',
    'PLZ': 'PLACA',
    'RDG': 'RIDGE',
    'SQ': 'SQUARE',
    'STA': 'STATION',
    'TER': 'TERRACE',
    'TRL': 'TRAIL',
    'TPKE': 'TURNPIKE',
    'VLY': 'VALLEY',
    'IS': 'ISLAND'
}


def translate_addr(addr_string):
    addr_string = addr_string.upper()
    new_addr = []
    
    tokens = addr_string.split()
    
    for t in tokens:
        if t in addr_translate:
            new_addr.append(addr_translate[t])
        else:
            new_addr.append(t)
    
    return ' '.join(new_addr)

def analyze_numbers(dataframe):
    
    address_match = 0
    city_match = 0
    zipcode_match = 0
    healthcount = 0
    mailcount = 0
    connectcount = 0
    portcount = 0
    unknowncount = 0
    disconnectedcount = 0
    voipcount = 0
    wirelesscount = 0
    no_address_count = 0
    no_date_count = 0
    business_count = 0
    residential_count = 0
    low_count = 0
    high_count = 0
    med_count = 0
    first_name_match = 0
    last_name_match = 0 
    workplace_match = 0
    state_match = 0
    no_name = 0
    SIC_count = 0
        


    for row in dataframe.itertuples():
        gpi_add = translate_addr(str(row.ContactAddressOut))
        ws_add = translate_addr(str(row.OFFICE_ADDRESS_LINE_2))
        if gpi_add != 'NONE':
            if gpi_add in ws_add or ws_add in gpi_add:
                address_match += 1
        if str(row.OFFICE_ADDRESS_CITY) in str(row.ContactCityOut):
            city_match += 1
        if clean_zipcode(row.OFFICE_ADDRESS_ZIP) == clean_zipcode(row.ContactZipOut):
            zipcode_match += 1
        
        if row.ContactStateOut == row.OFFICE_ADDRESS_STATE:
            state_match += 1
        
        if str(row.PHYSICIAN_LAST_NAME).lower() in str(row.ContactNameOut).lower():
            last_name_match +=1
        if str(row.PHYSICIAN_FIRST_NAME).lower() in str(row.ContactNameOut).lower():
            first_name_match +=1
        
        if str(row.OFFICE_ADDRESS_LINE_1).lower() in str(row.ContactNameOut).lower():
            workplace_match += 1

        if is_healthy(row.ContactNameOut)==True:    
            healthcount += 1

        if 'IsMailable' in str(row.NoteDescriptions):
            mailcount += 1
        if 'IsConnected' in str(row.NoteDescriptions):
            connectcount += 1
        if 'IsPorted' in str(row.NoteDescriptions):
            portcount += 1
        if 'IsUnknownContact' in str(row.NoteDescriptions):
            unknowncount += 1
        if 'IsPossibleDisconnected' in str(row.NoteDescriptions):
            disconnectedcount += 1
        if 'IsPossiblePortableVOIP' in str(row.NoteDescriptions):
            voipcount += 1
        if 'IsWireless' in str(row.NoteDescriptions):
            wirelesscount += 1

        if row.ContactAddressOut == 'None':
            no_address_count += 1
        if row.DateOfPorting == 'None':
            no_date_count += 1
        if row.ContactPhoneType == 'BUSINESS':
            business_count += 1
        if row.ContactPhoneType == 'RESIDENTIAL':
            residential_count += 1
        if row.ContactQualityScore == 'LOW':
            low_count += 1
        if row.ContactQualityScore == 'HIGH':
            high_count += 1
        if row.ContactQualityScore == 'MED':
            med_count += 1
        if row.ContactNameOut =='None':
            no_name += 1
#         if row.ContactSICDesc == 'OFFICES AND CLINICS OF MEDICAL DOCTORS':
#             SIC_count += 1
    other_dict = {
        'Address Matches': address_match,
        'City Matches': city_match,
        'ZipCode Matches': zipcode_match,
        'No Address': no_address_count,
        'No Date': no_date_count,
        'Relevant Name': healthcount,
        'Business Phone': business_count,
        'Residential Phone': residential_count,
        'Low Quality': low_count,
        'Medium Quality': med_count,
        'High Quality': high_count,
        'Mailable': mailcount,
        'Connected': connectcount,
        'Ported': portcount,
        'Unknown Contact': unknowncount,
        'Possibly Disconnected': disconnectedcount,
        'Possibly Portable VOIP': voipcount,
        'Wireless': wirelesscount,
        'First Name Match':first_name_match,
        'Last Name Match':last_name_match,
        'Workplace Match': workplace_match,
        'State Match': state_match, 
        'No Name': no_name,
        'Relevant SIC': SIC_count
        }
    new_dict = {
        'Address Matches': address_match/189*100,
        'City Matches': city_match/189*100,
        'ZipCode Matches': zipcode_match/189*100,
        'No Address': no_address_count/189*100,
        'No Date': no_date_count/189*100,
        'Relevant Name': healthcount/189*100,
        'Business Phone': business_count/189*100,
        'Residential Phone': residential_count/189*100,
        'Low Quality': low_count/189*100,
        'Medium Quality': med_count/189*100,
        'High Quality': high_count/189*100,
        'Mailable': mailcount/189*100,
        'Connected': connectcount/189*100,
        'Ported': portcount/189*100,
        'Unknown Contact': unknowncount/189*100,
        'Possibly Disconnected': disconnectedcount/189*100,
        'Possibly Portable VOIP': voipcount/189*100,
        'Wireless': wirelesscount/189*100,
        'First Name Match':first_name_match/189*100,
        'Last Name Match':last_name_match/189*100,
        'Workplace Match': workplace_match/189*100,
        'State Match': state_match/189*100, 
        'No Name': no_name/189*100,
        'Relevant SIC': SIC_count
        }
        
    return(new_dict, other_dict)

In [20]:
dict_1, dict_2 = analyze_numbers(wrong)

In [25]:
list_=[dict_1, dict_2]

In [27]:
pd.DataFrame(list_).transpose()

,0,1
Address Matches,24.867725,47.0
City Matches,40.740741,77.0
ZipCode Matches,34.920635,66.0
No Address,19.576720,37.0
No Date,33.862434,64.0
Relevant Name,71.428571,135.0
Business Phone,87.830688,166.0
Residential Phone,8.994709,17.0
Low Quality,31.746032,60.0
Medium Quality,7.936508,15.0
